# The fonctions to import


In [1]:
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette('deep')



# The scraping of the funds of french private company's

In [2]:


def scrape_french_funds():
    url = 'https://www.investing.com/funds/france-funds'
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        Name = []
        Symbol = []
        Last = []
        Chg = []
        Total_Assets = []
        Time = []

        table = soup.find('table')
        if table:
            rows = table.find_all('tr')[1:]  
            for row in rows:
                cols = row.find_all('td')
                # print(f"Columns in row: {[col.text.strip() for col in cols]}") # DEBUG PRINT: Show column contents
                if len(cols) >= 8:  
                    Name.append(cols[1].text.strip())  
                    Symbol.append(cols[2].text.strip()) 
                    Last.append(cols[3].text.strip())   
                    Total_Assets.append(cols[5].text.strip()) 
                    Time.append(cols[6].text.strip())   
                    try:
                        Chg_value = float(cols[4].text.strip().replace('%', '')) 
                        Chg.append(Chg_value)
                    except ValueError:
                        Chg.append(None)
                        print(f"Warning: Could not convert '{cols[4].text.strip()}' to float for 'Chg'")

        # Create DataFrame
        df = pd.DataFrame({
            'Name': Name,
            'Symbol': Symbol,
            'Last': Last,
            'Chg': Chg,
            'Total_Assets': Total_Assets,
            'Time': Time
            })

        return df

    except Exception as e:
        print(f'Error scraping data: {e}')
        return None

df = scrape_french_funds()
print(df)

Error scraping data: 403 Client Error: Forbidden for url: https://www.investing.com/funds/france-funds
None


# Fetch the data


In [3]:
funds_df = scrape_french_funds()

if funds_df is not None:
    print('French Companies Funds Data:')
    display(funds_df)

Error scraping data: 403 Client Error: Forbidden for url: https://www.investing.com/funds/france-funds


# Risk Analysis

In [4]:
funds_df['Risk'] = funds_df['Chg'].abs()
sorted_df = funds_df.sort_values(by='Risk', ascending=True)

print("Funds from Lowest to Highest Risk:")
display(sorted_df[['Name', 'Risk', 'Total_Assets', 'Chg']])

TypeError: 'NoneType' object is not subscriptable